In [1]:
import pandas as pd
from pydataxm import *
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer, Date, Float  # Importar tipos necesarios
import  os
import psycopg2

In [2]:
host = os.environ.get("POSTGRES_HOST")
port = int(os.environ.get("POSTGRES_PORT"))
database = os.environ.get("POSTGRES_DB")
user = os.environ.get("POSTGRES_USER")
password = os.environ.get("POSTGRES_PASSWORD")

# Crear el engine de SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [3]:
#Definir periodo de los datos
today = dt.date.today()
month_ago = today - dt.timedelta(days=360) # Approximation of a month

In [4]:
#Conexion a la API
objetoAPI = pydataxm.ReadDB()

**GENERACIÓN**

In [5]:
df_Generacion = objetoAPI.request_data(
                        "Gene",           #Se indica el nombre de la métrica tal como se llama en el campo metricId
                        "Recurso",               #Se indica el nombre de la entidad tal como se llama en el campo Entity
                        month_ago,     #Corresponde a la fecha inicial de la consulta
                        today)


df_Generacion.rename(columns={'Values_code': 'pkRecurso'}, inplace=True)


# Assume df_filtered is already defined from the preceding code

# Identify columns to pivot
value_hour_cols = [col for col in df_Generacion.columns if col.startswith('Values_Hour')]

# Melt the DataFrame
df_Generacion = pd.melt(df_Generacion,
                   id_vars=[col for col in df_Generacion.columns if not col.startswith('Values_Hour')],
                   value_vars=value_hour_cols,
                   var_name='hora',
                   value_name='generacion')

# Clean up the 'hora' column to extract just the hour number
df_Generacion['hora'] = df_Generacion['hora'].str.replace('Values_Hour', '').astype(int)

df_Generacion["fecha_actualizacion"] = today

df_Generacion=df_Generacion[['pkRecurso','Date','hora','generacion','fecha_actualizacion']]
# Display the pivoted DataFrame
df_Generacion.head()


,pkRecurso,Date,hora,generacion,fecha_actualizacion
0,2QBW,2024-06-07,1,16081.86,2025-06-02
1,2QEK,2024-06-07,1,35208.86,2025-06-02
2,2QRL,2024-06-07,1,640.68,2025-06-02
3,2QV2,2024-06-07,1,18752.40,2025-06-02
4,2R22,2024-06-07,1,17928.87,2025-06-02


In [7]:
# Insertar datos en PostgreSQL (modo 'append' para no borrar datos existentes)
with engine.begin() as conn:  # <- Se hace COMMIT automático al final
    df_Generacion.to_sql(
    name='generacion',          # Nombre de la tabla (sin esquema aquí)
    con=engine,
    schema='tablasxm_dev',       # Esquema especificado aquí
    if_exists='replace',          # Opción: 'append', 'replace' o 'fail'
    index=True,                 # Incluir el índice del DataFrame
    dtype={
           'pkrecurso': String(),     
           'date': Date(),
           'hora': Integer(),
           'generacion': Float(),              
           'fecha_actualizacion': Date()
    })



print("Datos insertados correctamente.")

Datos insertados correctamente.


In [8]:
print(len(df_Generacion))

2565504
